## Check metric polatiry

This notebook acesses the evaluation-tables.json file and compares the rank1 and rank 2 values for all the metrics.

This outputs a file called df_metric_all.csv used for the calculation of the polarity, comparing the values of the ranks 1 and 2 per metric and dataset.

Make sure the file evaluation-tables.json is available in the working dir.


In [4]:
#Adjust display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [5]:
#Load modules
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
from SPARQLWrapper import SPARQLWrapper, N3, JSON
from rdflib import Graph
import plotly.graph_objects as go
import plotly.express as px
import re

#setup some pandas display modes
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 5)
pd.set_option('display.max_colwidth', None)

%matplotlib inline

In [6]:
#read evaluation table
df = pd.DataFrame(pd.read_json("evaluation-tables.json"))


In [7]:
#check the json
pd.set_option("display.max_rows", 5)
pd.set_option('display.max_colwidth', None)

#this is once specific category (100)
df.iloc[100].subtasks

[{'categories': [],
  'datasets': [{'dataset': 'SYSU 3D',
    'dataset_citations': [],
    'dataset_links': [{'title': 'Papers with Code Leaderboard URL',
      'url': 'https://paperswithcode.com/sota/skeleton-based-action-recognition-on-sysu-3d'}],
    'description': '',
    'sota': {'metrics': ['Accuracy'],
     'rows': [{'code_links': [{'title': 'microsoft/SGN',
         'url': 'https://github.com/microsoft/SGN'}],
       'metrics': {'Accuracy': '86.9%'},
       'model_links': [],
       'model_name': 'SGN',
       'paper_date': '2019-04-02',
       'paper_title': 'Semantics-Guided Neural Networks for Efficient Skeleton-Based Human Action Recognition',
       'paper_url': 'https://arxiv.org/abs/1904.01189v3',
       'uses_additional_data': False},
      {'code_links': [{'title': 'microsoft/View-Adaptive-Neural-Networks-for-Skeleton-based-Human-Action-Recognition',
         'url': 'https://github.com/microsoft/View-Adaptive-Neural-Networks-for-Skeleton-based-Human-Action-Recognition'

In [8]:
#get a sample (category) at index == 100
sample= df[df.index==100]

In [9]:
#display it
sample

categories  \
100  [Computer Vision]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [10]:
#get all the subtasks of this sample
subtasks = sample["subtasks"].iloc[0]

len(subtasks)

3

In [11]:
#print different tasks
print(subtasks[0]["task"])
print(subtasks[1]["task"])
print(subtasks[2]["task"])

Skeleton Based Action Recognition
One-Shot 3D Action Recognition
Mutual Gaze


In [12]:
#get one subtask
subtasks[0]

{'categories': [],
 'datasets': [{'dataset': 'SYSU 3D',
   'dataset_citations': [],
   'dataset_links': [{'title': 'Papers with Code Leaderboard URL',
     'url': 'https://paperswithcode.com/sota/skeleton-based-action-recognition-on-sysu-3d'}],
   'description': '',
   'sota': {'metrics': ['Accuracy'],
    'rows': [{'code_links': [{'title': 'microsoft/SGN',
        'url': 'https://github.com/microsoft/SGN'}],
      'metrics': {'Accuracy': '86.9%'},
      'model_links': [],
      'model_name': 'SGN',
      'paper_date': '2019-04-02',
      'paper_title': 'Semantics-Guided Neural Networks for Efficient Skeleton-Based Human Action Recognition',
      'paper_url': 'https://arxiv.org/abs/1904.01189v3',
      'uses_additional_data': False},
     {'code_links': [{'title': 'microsoft/View-Adaptive-Neural-Networks-for-Skeleton-based-Human-Action-Recognition',
        'url': 'https://github.com/microsoft/View-Adaptive-Neural-Networks-for-Skeleton-based-Human-Action-Recognition'},
       {'title'

In [13]:
#count how many datasets
len(subtasks[0]["datasets"])

29

In [14]:
#get the same of the dataset
subtasks[0]["datasets"][0]["dataset"]

'SYSU 3D'

In [15]:
#this is all the data for such dataset
subtasks[0]["datasets"][0]

{'dataset': 'SYSU 3D',
 'dataset_citations': [],
 'dataset_links': [{'title': 'Papers with Code Leaderboard URL',
   'url': 'https://paperswithcode.com/sota/skeleton-based-action-recognition-on-sysu-3d'}],
 'description': '',
 'sota': {'metrics': ['Accuracy'],
  'rows': [{'code_links': [{'title': 'microsoft/SGN',
      'url': 'https://github.com/microsoft/SGN'}],
    'metrics': {'Accuracy': '86.9%'},
    'model_links': [],
    'model_name': 'SGN',
    'paper_date': '2019-04-02',
    'paper_title': 'Semantics-Guided Neural Networks for Efficient Skeleton-Based Human Action Recognition',
    'paper_url': 'https://arxiv.org/abs/1904.01189v3',
    'uses_additional_data': False},
   {'code_links': [{'title': 'microsoft/View-Adaptive-Neural-Networks-for-Skeleton-based-Human-Action-Recognition',
      'url': 'https://github.com/microsoft/View-Adaptive-Neural-Networks-for-Skeleton-based-Human-Action-Recognition'},
     {'title': 'iamjeff7/j-va-aagcn',
      'url': 'https://github.com/iamjeff7/

In [16]:
#show list of metrics per dataset
subtasks[0]["datasets"][0]["sota"]["metrics"]

['Accuracy']

In [17]:
#show metric result for rank 1 (["rows"][0])
subtasks[0]["datasets"][0]["sota"]["rows"][0]["metrics"]

{'Accuracy': '86.9%'}

In [18]:
#show metric result for rank 2 (["rows"][1])
subtasks[0]["datasets"][0]["sota"]["rows"][1]["metrics"]

{'Accuracy': '86.7%'}

In [19]:
#sometimes it is just reported one metric 
subtasks[1]["datasets"][0]["sota"]["rows"][0]["metrics"]

{'Accuracy': '49.6%'}

### create a function to retrieve all metrics of a subtasks

In [20]:
def get_all_metrics(subtasks):
    #create report data frame for the subtasks of the taken sample
    df_metric_all = pd.DataFrame(columns=["value","metrics","ranking","datasets","task","categories"])
    #df_metric_all

    s = 0
    while s < len(subtasks):
        d = 0
        while d < len(subtasks[s]["datasets"]):
            #iterate over the different metrics of a dataset
            m = 0
            while m < len(subtasks[s]["datasets"][d]["sota"]["rows"]):
                df_metric = pd.DataFrame.from_dict(subtasks[s]["datasets"][d]["sota"]["rows"][m]["metrics"], orient='index')
                df_metric.rename(columns = {0: 'value'}, inplace = True)
                df_metric["metrics"]=df_metric.index
                df_metric["ranking"]= m + 1
                df_metric["datasets"] = subtasks[s]["datasets"][d]["dataset"]
                df_metric["task"]= subtasks[s]["task"]
                df_metric["categories"]= str(sample["categories"].iloc[0])
                df_metric_all = df_metric_all.append(df_metric, ignore_index=True)
                m = m +1 
            d = d + 1
        s = s + 1

    #["value","metric","ranking","datasets","task"])    
    df_metric_all = df_metric_all[['categories','task', 'datasets', 'metrics','ranking', 'value']]

    df_metric_all["value"] = df_metric_all["value"].replace(to_replace ='\%', value = '', regex = True)
    df_metric_all["value"] = df_metric_all["value"].replace(to_replace ='\-', value = '', regex = True)
    df_metric_all["value"] = df_metric_all["value"].replace(to_replace =' .*', value = '', regex = True)
    df_metric_all["value"] = df_metric_all["value"].replace(to_replace =',', value = '', regex = True)


    df_metric_all["categories"] = df_metric_all["categories"].replace(to_replace ='\[\'', value = '', regex = True)
    df_metric_all["categories"] = df_metric_all["categories"].replace(to_replace ='\'\]', value = '', regex = True)
    
    return(df_metric_all)

In [21]:
#how many categories
len(df.index)

1183

## Attention!!

This is the block that creates the file df_metric_all.csv required by the "Complete collapsed workflow" notebook.

Change this block to code and run it if the file is not available.

In [22]:
#reads pre-saved results for all the categories
df_metric_all = pd.read_csv("df_metric_all.csv")
df_metric_all.head(4)

,categories,task,datasets,metrics,ranking,value
0,"Computer Vision', 'Natural Language Processing",Handwriting Recognition,BanglaLekha Isolated Dataset,Accuracy,1,96.8
1,"Computer Vision', 'Natural Language Processing",Handwriting Recognition,BanglaLekha Isolated Dataset,Cross Entropy Loss,1,0.21612
2,"Computer Vision', 'Natural Language Processing",Handwriting Recognition,BanglaLekha Isolated Dataset,Epochs,1,11
3,"Computer Vision', 'Natural Language Processing",Handwritten Digit Recognition,MNIST,Accuracy,1,96.95


In [23]:
df_metric_all["categories"].unique()

array(["Computer Vision', 'Natural Language Processing",
       "Computer Vision', 'Miscellaneous', 'Time Series",
       'Computer Vision', 'Time Series', "Time Series', 'Methodology",
       "Medical', 'Knowledge Base", "Computer Vision', 'Methodology",
       'Speech', "Speech', 'Natural Language Processing",
       'Natural Language Processing', "Computer Vision', 'Time Series",
       "Medical', 'Computer Vision",
       "Computer Vision', 'Miscellaneous', 'Methodology", 'Methodology',
       "Computer Code', 'Natural Language Processing", 'Knowledge Base',
       "Robots', 'Computer Vision", 'Medical', 'Miscellaneous',
       "Robots', 'Miscellaneous",
       "Computer Vision', 'Natural Language Processing', 'Speech', 'Miscellaneous",
       'Audio', "Medical', 'Methodology",
       "Miscellaneous', 'Computer Vision",
       "Computer Vision', 'Playing Games", "Computer Code', 'Reasoning",
       "Miscellaneous', 'Methodology", 'Graphs', 'Playing Games',
       "Computer Vision',

In [24]:
#Keep only rakns 1 and 2
#Filter out strange metrics

df_metric_all = df_metric_all[df_metric_all["ranking"]<=2]
df_metric_all = df_metric_all[df_metric_all["ranking"]>0]
df_metric_all = df_metric_all[df_metric_all["metrics"]!="No. parameters"]

In [25]:
#calculate polarity (takes 1-2 min to run)
#report df
polarity_df = pd.DataFrame(columns=['task','datasets','metricName','rank_1', 'rank_2', 'polarity'])

#iterate over tasks
for task in df_metric_all["task"].unique():
    #iterate over datasets
    for dataset in df_metric_all[df_metric_all["task"]==task]["datasets"].unique():
        #iterate over metrics
        for metric in df_metric_all[df_metric_all["task"]==task]["metrics"].unique():

            #get ranks 1 and 2, compare and set polarity
            try:
                rank_1 = df_metric_all[(df_metric_all["task"] == task) & (df_metric_all["datasets"] == dataset) & (df_metric_all["metrics"] == metric ) & (df_metric_all["ranking"] == 1 )]
                rank_2 = df_metric_all[(df_metric_all["task"] == task) & (df_metric_all["datasets"] == dataset) & (df_metric_all["metrics"] == metric ) & (df_metric_all["ranking"] == 2 )]
                polarity = float(rank_1.value.iloc[0]) - float(rank_2.value.iloc[0])
                if(polarity >=0):
                    polarity = "pos"
                else:
                    polarity = "neg"

                print("metricName:"+metric + "\trank_1:"+rank_1["value"].iloc[0]+"\trank_2:"+rank_2["value"].iloc[0]+"\tpolarity:"+polarity)
                
                #save to report df
                polarity_df = polarity_df.append({'task':task,
                                                  'datasets':dataset,
                                                  'metricName':metric,
                                                  'rank_1':rank_1["value"].iloc[0],
                                                  'rank_2':rank_2["value"].iloc[0],
                                                  'polarity':polarity},
                                                  ignore_index=True)
            #skip if metric is not present for such dataset    
            except:
                next

    

metricName:COL	rank_1:5.31	rank_2:6.560	polarity:neg
metricName:FDE	rank_1:1.14	rank_2:1.150	polarity:neg
metricName:absolute relative error	rank_1:0.079	rank_2:0.087	polarity:neg
metricName:Abs Rel	rank_1:0.377	rank_2:0.322	polarity:pos
metricName:RMSE	rank_1:8.388	rank_2:7.417	polarity:pos
metricName:Sq Rel	rank_1:4.9	rank_2:3.589	polarity:pos
metricName:D3R	rank_1:0.3222	rank_2:0.4671	polarity:neg
metricName:ORD	rank_1:0.3938	rank_2:0.5538	polarity:neg
metricName:RMSE	rank_1:0.357	rank_2:0.364	polarity:neg
metricName:RMSE	rank_1:0.1557	rank_2:0.1973	polarity:neg
metricName:D3R	rank_1:0.1578	rank_2:0.2324	polarity:neg
metricName:δ1.25	rank_1:0.7406	rank_2:0.7891	polarity:neg
metricName:ORD 	rank_1:0.3467	rank_2:0.3879	polarity:neg
metricName:absolute relative error	rank_1:0.058	rank_2:0.059	polarity:neg
metricName:Abs Rel	rank_1:0.1425	rank_2:0.2410	polarity:neg
metricName:RMSE	rank_1:8.8641	rank_2:12.599	polarity:neg
metricName:RMSE log	rank_1:0.24571	rank_2:0.3618	polarity:neg
metr

metricName:Overall Accuracy	rank_1:92.05	rank_2:89.075	polarity:pos
metricName:pose	rank_1:1.12	rank_2:1.22	polarity:neg
metricName:NME	rank_1:3.13	rank_2:3.24	polarity:neg
metricName:Mean NME 	rank_1:1.91	rank_2:2.17	polarity:neg
metricName:AUC0.08 private	rank_1:59.39	rank_2:58.22	polarity:pos
metricName:Mean Reconstruction Error (mm)	rank_1:1.91	rank_2:2.08	polarity:neg
metricName:Mean NME 	rank_1:3.56	rank_2:3.9625	polarity:neg
metricName:Mean NME 	rank_1:3.56	rank_2:3.7551	polarity:neg
metricName:Mean Reconstruction Error (mm)	rank_1:1.89	rank_2:1.91	polarity:neg
metricName:Mean Reconstruction Error (mm)	rank_1:1.38	rank_2:1.53	polarity:neg
metricName:Accuracy	rank_1:73.56	rank_2:61.80	polarity:pos
metricName:Accuracy	rank_1:94.60	rank_2:91.4	polarity:pos
metricName:Accuracy	rank_1:99.10	rank_2:98.78	polarity:pos
metricName:EER	rank_1:4.92	rank_2:2.22	polarity:pos
metricName:EER	rank_1:2.14	rank_2:0.40	polarity:pos
metricName:Equal Error Rate	rank_1:7.5	rank_2:10.8	polarity:neg
me

metricName:Avg F1	rank_1:19.77	rank_2:19.09	polarity:pos
metricName:BLEU	rank_1:14.4	rank_2:14.79	polarity:neg
metricName:Entity F1	rank_1:62.7	rank_2:59.97	polarity:pos
metricName:METEOR	rank_1:0.331	rank_2:0.089	polarity:pos
metricName:Joint	rank_1:75.5	rank_2:74.5	polarity:pos
metricName:Joint	rank_1:90.5	rank_2:88.9	polarity:pos
metricName:MRR	rank_1:0.7124	rank_2:0.7041	polarity:pos
metricName:Mean Rank	rank_1:2.96	rank_2:3.66	polarity:neg
metricName:R@1	rank_1:58.28	rank_2:58.18	polarity:pos
metricName:R@10	rank_1:94.45	rank_2:90.83	polarity:pos
metricName:R@5	rank_1:87.55	rank_2:83.85	polarity:pos
metricName:MRR	rank_1:68.92	rank_2:66.38	polarity:pos
metricName:Mean Rank	rank_1:3.39	rank_2:4.04	polarity:neg
metricName:R@1	rank_1:55.16	rank_2:53.33	polarity:pos
metricName:R@10	rank_1:92.95	rank_2:90.38	polarity:pos
metricName:R@5	rank_1:86.26	rank_2:82.42	polarity:pos
metricName:R@1	rank_1:44.45	rank_2:44.75	polarity:neg
metricName:R@10	rank_1:83.78	rank_2:82.75	polarity:pos
metr

metricName:Kernel Inception Distance	rank_1:10.23	rank_2:11.52	polarity:neg
metricName:mIoU	rank_1:63.3	rank_2:59.6	polarity:pos
metricName:Class IOU	rank_1:0.26	rank_2:0.09	polarity:pos
metricName:Per-class Accuracy	rank_1:46	rank_2:22	polarity:pos
metricName:Per-pixel Accuracy	rank_1:70	rank_2:42	polarity:pos
metricName:mIoU	rank_1:57.5	rank_2:49.9	polarity:pos
metricName:classification score	rank_1:78.1	rank_2:59.4	polarity:pos
metricName:FID	rank_1:34.4	rank_2:38.1	polarity:neg
metricName:DFID	rank_1:35.6	rank_2:56.2	polarity:neg
metricName:LPIPS	rank_1:0.505	rank_2:0.43	polarity:pos
metricName:FID	rank_1:13.73	rank_2:14.3	polarity:neg
metricName:FID	rank_1:48.6	rank_2:63.3	polarity:neg
metricName:mIoU	rank_1:40.4	rank_2:30.8	polarity:pos
metricName:Kernel Inception Distance	rank_1:11.40	rank_2:11.61	polarity:neg
metricName:Classification Error	rank_1:2.12	rank_2:4.10	polarity:neg
metricName:FID	rank_1:17.0	rank_2:19.1	polarity:neg
metricName:mIoU	rank_1:44.1	rank_2:42.1	polarity:p

metricName:R@1	rank_1:4.34	rank_2:2.70	polarity:pos
metricName:R@10	rank_1:13.97	rank_2:8.93	polarity:pos
metricName:R@100	rank_1:21.78	rank_2:15.34	polarity:pos
metricName:text-to-video Median Rank	rank_1:4	rank_2:9	polarity:neg
metricName:text-to-video R@1	rank_1:28.9	rank_2:16.7	polarity:pos
metricName:text-to-video R@10	rank_1:70.0	rank_2:52.3	polarity:pos
metricName:text-to-video Mean Rank	rank_1:58.0	rank_2:58.0	polarity:pos
metricName:text-to-video Median Rank	rank_1:11.0	rank_2:12.3	polarity:neg
metricName:text-to-video R@1	rank_1:21.6	rank_2:18.8	polarity:pos
metricName:text-to-video R@10	rank_1:49.8	rank_2:47.9	polarity:pos
metricName:text-to-video R@5	rank_1:41.8	rank_2:38.5	polarity:pos
metricName:text-to-video Median Rank	rank_1:2.0	rank_2:3	polarity:neg
metricName:text-to-video R@1	rank_1:43.4	rank_2:31.0	polarity:pos
metricName:text-to-video R@10	rank_1:80.6	rank_2:72.4	polarity:pos
metricName:text-to-video R@5	rank_1:70.2	rank_2:59.8	polarity:pos
metricName:text-to-vide

...

metricName:Micro-F1	rank_1:68.555	rank_2:64.1	polarity:pos
metricName:Average F1	rank_1:0.8887	rank_2:0.8246	polarity:pos
metricName:Weighted F1	rank_1:0.8634	rank_2:0.8231	polarity:pos
metricName:Average F1	rank_1:0.8882	rank_2:0.6642	polarity:pos
metricName:Weighted F1	rank_1:0.9072	rank_2:0.8137	polarity:pos
metricName:P@5	rank_1:68.7	rank_2:52.83	polarity:pos
metricName:nDCG@5	rank_1:82.3	rank_2:68.8	polarity:pos
metricName:Average F1	rank_1:0.8843	rank_2:0.7761	polarity:pos
metricName:Weighted F1	rank_1:0.8957	rank_2:0.8235	polarity:pos
metricName:Accuracy (%)	rank_1:57.9	rank_2:56.3	polarity:pos
metricName:Accuracy (%)	rank_1:82.1	rank_2:80.2	polarity:pos
metricName:F1	rank_1:67.9	rank_2:54.6	polarity:pos
metricName:F1	rank_1:84.99	rank_2:84.0	polarity:pos
metricName:Accuracy	rank_1:55.39	rank_2:55.09	polarity:pos
metricName:Average F1	rank_1:46.98	rank_2:46.65	polarity:pos
metricName:NME	rank_1:6.31	rank_2:6.54	polarity:neg
metricName:NME	rank_1:7.53	rank_2:10.53	polarity:neg
me

metricName:AED	rank_1:0.133	rank_2:0.134	polarity:neg
metricName:AKD	rank_1:1.28	rank_2:1.27	polarity:pos
metricName:L1	rank_1:0.040	rank_2:0.041	polarity:neg
metricName:AED	rank_1:0.172	rank_2:0.203	polarity:neg
metricName:AKD	rank_1:13.86	rank_2:17.12	polarity:neg
metricName:L1	rank_1:0.064	rank_2:0.075	polarity:neg
metricName:MKR	rank_1:0.043	rank_2:0.066	polarity:neg
metricName:Rank-1 Recognition Rate	rank_1:98.2	rank_2:85.45	polarity:pos
metricName:Average Recall	rank_1:99.18	rank_2:98.36	polarity:pos
metricName:GFLOPs	rank_1:0.34	rank_2:0.94	polarity:neg
metricName:Rank-1 Recognition Rate	rank_1:90	rank_2:64.89	polarity:pos
metricName:mIoU	rank_1:47.3	rank_2:35.8	polarity:pos
metricName:Accuracy (Test)	rank_1:71.56	rank_2:70.61	polarity:pos
metricName:Accuracy (%)	rank_1:89.4	rank_2:89.0	polarity:pos
metricName:Percentage error	rank_1:1.6	rank_2:1.8	polarity:neg
metricName:Percentage Error	rank_1:11.7	rank_2:12.3	polarity:neg
metricName:Top-1 Error Rate	rank_1:1.98	rank_2:2.0	pol

metricName:S-Measure	rank_1:0.856	rank_2:0.819	polarity:pos
metricName:S-Measure	rank_1:0.619	rank_2:0.608	polarity:pos
metricName:Average MAE	rank_1:0.114	rank_2:0.131	polarity:neg
metricName:max E-measure	rank_1:0.696	rank_2:0.698	polarity:neg
metricName:S-Measure	rank_1:0.942	rank_2:0.907	polarity:pos
metricName:Average MAE	rank_1:0.021	rank_2:0.032	polarity:neg
metricName:max E-measure	rank_1:0.980	rank_2:0.846	polarity:pos
metricName:AVERAGE MAE	rank_1:0.040	rank_2:0.054	polarity:neg
metricName:S-Measure	rank_1:0.879	rank_2:0.870	polarity:pos
metricName:MAX F-MEASURE	rank_1:0.865	rank_2:0.861	polarity:pos
metricName:S-Measure	rank_1:0.755	rank_2:0.706	polarity:pos
metricName:Average MAE	rank_1:0.084	rank_2:0.114	polarity:neg
metricName:max E-Measure	rank_1:0.806	rank_2:0.749	polarity:pos
metricName:max F-Measure	rank_1:0.659	rank_2:0.591	polarity:pos
metricName:S-Measure	rank_1:0.901	rank_2:0.860	polarity:pos
metricName:Average MAE	rank_1:0.018	rank_2:0.025	polarity:neg
metricName

metricName:AP@0.15	rank_1:44.91	rank_2:39.09	polarity:pos
metricName:AP Hard	rank_1:11.46	rank_2:9.94	polarity:pos
metricName:MRR	rank_1:0.493	rank_2:0.402	polarity:pos
metricName:Hits@1	rank_1:0.289	rank_2:0.222	polarity:pos
metricName:Avg	rank_1:81.7	rank_2:81.6	polarity:pos
metricName:Question Answering	rank_1:71.9	rank_2:72.5	polarity:neg
metricName:Sentence Retrieval	rank_1:90.8	rank_2:93.7	polarity:neg
metricName:Sentence-pair Classification	rank_1:88.3	rank_2:88.4	polarity:neg
metricName:Structured Prediction	rank_1:80.6	rank_2:76.2	polarity:pos
metricName:F1	rank_1:75.33	rank_2:74.97	polarity:pos
metricName:F1	rank_1:77.89	rank_2:76.42	polarity:pos
metricName:F1	rank_1:69.2	rank_2:68.9	polarity:pos
metricName:F1	rank_1:55.3	rank_2:50.89	polarity:pos
metricName:F1	rank_1:79.31	rank_2:78	polarity:pos
metricName:F1	rank_1:83.35	rank_2:82.9	polarity:pos
metricName:F-Measure (Seen)	rank_1:67.2	rank_2:60.5	polarity:pos
metricName:F-Measure (Unseen)	rank_1:51	rank_2:60.7	polarity:neg


metricName:Average PSNR	rank_1:34.80	rank_2:24.09	polarity:pos
metricName:PSNR (sRGB)	rank_1:39.31	rank_2:38.65	polarity:pos
metricName:SSIM (sRGB)	rank_1:0.972	rank_2:0.965	polarity:pos
metricName:PSNR (sRGB)	rank_1:35.99	rank_2:35.70	polarity:pos
metricName:SSIM (sRGB)	rank_1:0.952	rank_2:0.948	polarity:pos
metricName:PSNR (sRGB)	rank_1:30.96	rank_2:29.98	polarity:pos
metricName:SSIM (sRGB)	rank_1:0.939	rank_2:0.930	polarity:pos
metricName:PSNR	rank_1:32.71	rank_2:32.66	polarity:pos
metricName:PSNR	rank_1:32.34	rank_2:32.13	polarity:pos
metricName:PSNR (sRGB)	rank_1:31.76	rank_2:31.38	polarity:pos
metricName:SSIM (sRGB)	rank_1:0.922	rank_2:0.909	polarity:pos
metricName:PSNR (sRGB)	rank_1:28.70	rank_2:28.70	polarity:pos
metricName:SSIM (sRGB)	rank_1:0.873	rank_2:0.866	polarity:pos
metricName:Dice Score	rank_1:0.876	rank_2:0.831	polarity:pos
metricName:Dice Score	rank_1:0.840	rank_2:0.814	polarity:pos
metricName:Precision	rank_1:0.849	rank_2:0.848	polarity:pos
metricName:Recall	rank_1:

In [26]:
#check the polarity data frame. (Results from above)
polarity_df.head()

,task,datasets,metricName,rank_1,rank_2,polarity
0,Trajectory Forecasting,TrajNet++,COL,5.31,6.560,neg
1,Trajectory Forecasting,TrajNet++,FDE,1.14,1.150,neg
2,Monocular Depth Estimation,KITTI Eigen split unsupervised,absolute relative error,0.079,0.087,neg
3,Monocular Depth Estimation,Make3D,Abs Rel,0.377,0.322,pos
4,Monocular Depth Estimation,Make3D,RMSE,8.388,7.417,pos


In [27]:
#display the summary of neg and pos counts for every metric:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)

#summarise
polarity_df_report = pd.DataFrame(polarity_df.groupby(['metricName', 'polarity'])['datasets'].count())
#order
polarity_df_report = polarity_df_report.sort_index(ascending=True)
#display
polarity_df_report
#Note: this is basically the number of times every metric was positive or negative.

,,datasets
metricName,polarity,
RMSE (Subject-exposed),neg,1
three pixel error,neg,1
# of clusters (k),pos,1
% Test Accuracy,pos,1
0..5sec,neg,1
1-1,pos,2
10 sec,pos,1
10%,pos,1
10-20% Mask PSNR,pos,1


In [28]:
#metrics with negative polarity    
polarity_df[polarity_df["polarity"]=="neg"]["metricName"].unique()

array(['COL', 'FDE', 'absolute relative error', 'D3R', 'ORD', 'RMSE',
       'δ1.25', 'ORD ', 'Abs Rel', 'RMSE log', 'SQ Rel',
       ' three pixel error', 'SSIM', 'NIQE', 'FID', 'PSNR', 'FED',
       'LPIPS', 'Fullset (public)', 'Mean Error Rate', 'Mean NME ',
       'Mean NME', 'Error rate', 'pose', 'NME',
       'Mean Reconstruction Error (mm)', 'Equal Error Rate', 'MAE',
       'NLDA', 'GAR @0.1% FAR Obfuscation', 'GAR @0.1% FAR Overall',
       'GAR @1% FAR Impersonation', 'GAR @1% FAR Obfuscation',
       'GAR @1% FAR Overall', 'Reasonable Miss Rate', 'Heavy MR^-2',
       'Partial MR^-2', 'Reasonable MR^-2', 'LCS', 'MIoU (13 classes)',
       'Quality', 'Kernel Inception Distance', 'Top 1 Error',
       'mean Corruption Error (mCE)', 'Top-1 Error Rate',
       'Word Error Rate (WER)', 'Percentage error', 'BLEU', 'Mean Rank',
       'R@1', 'Mean', 'MultiWOZ (Inform)', 'MultiWOZ (Success)',
       'Average MPJPE (mm)', 'MSE', 'MRPE', 'OOB Rate (10^−3) ',
       'Path Length', 'Ste

In [29]:
#define function
def get_all_negative_metrics():
    df_metric_all = pd.read_csv("df_metric_all.csv")

    df_metric_all = df_metric_all[df_metric_all["ranking"]<=2]
    df_metric_all = df_metric_all[df_metric_all["ranking"]>0]
    df_metric_all = df_metric_all[df_metric_all["metrics"]!="No. parameters"]

    #calculate polarity 
    #report df
    polarity_df = pd.DataFrame(columns=['task','datasets','metricName','rank_1', 'rank_2', 'polarity'])

    #iterate over tasks
    for task in df_metric_all["task"].unique():
        #iterate over datasets
        for dataset in df_metric_all[df_metric_all["task"]==task]["datasets"].unique():
            #iterate over metrics
            for metric in df_metric_all[df_metric_all["task"]==task]["metrics"].unique():

                #get ranks 1 and 2, compare and set polarity
                try:
                    rank_1 = df_metric_all[(df_metric_all["task"] == task) & (df_metric_all["datasets"] == dataset) & (df_metric_all["metrics"] == metric ) & (df_metric_all["ranking"] == 1 )]
                    rank_2 = df_metric_all[(df_metric_all["task"] == task) & (df_metric_all["datasets"] == dataset) & (df_metric_all["metrics"] == metric ) & (df_metric_all["ranking"] == 2 )]
                    polarity = float(rank_1.value.iloc[0]) - float(rank_2.value.iloc[0])
                    if(polarity >=0):
                        polarity = "pos"
                    else:
                        polarity = "neg"

                    #print("metricName:"+metric + "\trank_1:"+rank_1["value"].iloc[0]+"\trank_2:"+rank_2["value"].iloc[0]+"\tpolarity:"+polarity)

                    #save to report df
                    polarity_df = polarity_df.append({'task':task,
                                                      'datasets':dataset,
                                                      'metricName':metric,
                                                      'rank_1':rank_1["value"].iloc[0],
                                                      'rank_2':rank_2["value"].iloc[0],
                                                      'polarity':polarity},
                                                      ignore_index=True)
                #skip if metric is not present for such dataset    
                except:
                    next


    metricName_neg = polarity_df[polarity_df["polarity"]=="neg"]["metricName"].unique()
    
    return(metricName_neg)

In [30]:

#Show and export all the negative polarities.
metricName_negative = polarity_df[polarity_df["polarity"]=="neg"]["metricName"].unique()

#metricName_negative.tofile('metricName_neg_pol.csv', sep=",")

metricName_negative

array(['COL', 'FDE', 'absolute relative error', 'D3R', 'ORD', 'RMSE',
       'δ1.25', 'ORD ', 'Abs Rel', 'RMSE log', 'SQ Rel',
       ' three pixel error', 'SSIM', 'NIQE', 'FID', 'PSNR', 'FED',
       'LPIPS', 'Fullset (public)', 'Mean Error Rate', 'Mean NME ',
       'Mean NME', 'Error rate', 'pose', 'NME',
       'Mean Reconstruction Error (mm)', 'Equal Error Rate', 'MAE',
       'NLDA', 'GAR @0.1% FAR Obfuscation', 'GAR @0.1% FAR Overall',
       'GAR @1% FAR Impersonation', 'GAR @1% FAR Obfuscation',
       'GAR @1% FAR Overall', 'Reasonable Miss Rate', 'Heavy MR^-2',
       'Partial MR^-2', 'Reasonable MR^-2', 'LCS', 'MIoU (13 classes)',
       'Quality', 'Kernel Inception Distance', 'Top 1 Error',
       'mean Corruption Error (mCE)', 'Top-1 Error Rate',
       'Word Error Rate (WER)', 'Percentage error', 'BLEU', 'Mean Rank',
       'R@1', 'Mean', 'MultiWOZ (Inform)', 'MultiWOZ (Success)',
       'Average MPJPE (mm)', 'MSE', 'MRPE', 'OOB Rate (10^−3) ',
       'Path Length', 'Ste

Note: Some of the metrics display two polarities. This should be manually checked. For example see Top-5 Accuracy below:

In [31]:
#This example shows that on the polarity_df dataframe, the polarity of "Top-5 Accuracy" is wrong 
#on PWC.
polarity_df[polarity_df["metricName"]=="Top-5 Accuracy"]

,task,datasets,metricName,rank_1,rank_2,polarity
547,Action Classification,Kinetics-600,Top-5 Accuracy,97.3,96.5,pos
642,Action Recognition,Something-Something V2,Top-5 Accuracy,92.7,92.70,pos
716,Few-Shot Image Classification,ImageNet (1-shot),Top-5 Accuracy,58.2,59.2,neg
862,Scene Graph Generation,3R-Scan,Top-5 Accuracy,0.87,0.66,pos
1874,3D Object Classification,3R-Scan,Top-5 Accuracy,0.7,0.68,pos


In [32]:
#This filter shows where this metric is used.
df_metric_all[df_metric_all.metrics=="Top-5 Accuracy"]

,categories,task,datasets,metrics,ranking,value
1535,Computer Vision,Action Classification,Kinetics-600,Top-5 Accuracy,1,97.3
1537,Computer Vision,Action Classification,Kinetics-600,Top-5 Accuracy,2,96.5
1575,Computer Vision,Video Classification,Something-Something V2,Top-5 Accuracy,1,91
1576,Computer Vision,Video Classification,Something-Something V1,Top-5 Accuracy,1,84
1823,Computer Vision,Action Recognition,Something-Something V2,Top-5 Accuracy,1,92.7
1825,Computer Vision,Action Recognition,Something-Something V2,Top-5 Accuracy,2,92.70
1856,Computer Vision,Action Recognition,EgoGesture,Top-5 Accuracy,1,99.2
2032,"Computer Vision', 'Methodology",Few-Shot Image Classification,ImageNet (1-shot),Top-5 Accuracy,1,58.2
2033,"Computer Vision', 'Methodology",Few-Shot Image Classification,ImageNet (1-shot),Top-5 Accuracy,2,59.2
2440,Computer Vision,Scene Graph Generation,3R-Scan,Top-5 Accuracy,1,0.87


In [33]:
#count how many metrics are assigned to positive and negative polarities

In [34]:
polarity_df_report_2 = polarity_df_report.copy()

polarity_df_report_2.reset_index(inplace=True)  

polarity_df_report_2.head(5)

metrics_polarity_counts =  pd.DataFrame(polarity_df_report_2.groupby(['metricName'])['polarity'].count())

#count how many metrics have 2 different polarities
len(metrics_polarity_counts[metrics_polarity_counts["polarity"]==2])


82

In [35]:
#print these metrics
metrics_polarity_counts[metrics_polarity_counts["polarity"]==2].index

Index(['ACC@1-100Clients', 'AED', 'AKD', 'AVERAGE MAE', 'Abs Rel', 'Acc',
       'Accuracy (%)', 'Accuracy (Cross-Setup)', 'Average MAE', 'B1', 'B2',
       'B3', 'B4', 'BLEU', 'EER', 'EM', 'Edit', 'Error Rate', 'Exact Span F1',
       'F-Measure (Seen)', 'F-Measure (Unseen)', 'F-measure (Decay)', 'F1',
       'F1 score', 'F1-score', 'F1-score (Augmented)', 'F1@10%', 'F1@25%',
       'FID', 'GAR @0.1% FAR Obfuscation', 'GAR @0.1% FAR Overall', 'HR@20',
       'Hit@20', 'IS', 'Inception Score', 'IoU overall', 'Jaccard (Decay)',
       'Jaccard (Recall)', 'Jaccard (Unseen)', 'L1', 'LPIPS', 'MAE',
       'MAE [bpm, session-wise]', 'MAP', 'MAX E-MEASURE', 'MAX F-MEASURE',
       'METEOR', 'MKR', 'MPJPE', 'PCKh', 'PQth', 'PSNR', 'PSNR-B', 'Precision',
       'R-Prec', 'R@1', 'RMSE', 'ROUGE-2', 'ROUGE-L', 'Rank-5', 'Recall',
       'Restaurant 2014 (F1)', 'SPICE', 'SSIM', 'SSIM (sRGB)', 'Speed  (FPS)',
       'Top 5 Accuracy', 'Top-5 Accuracy', 'Video-mAP 0.5', 'bits/dimension',
       'mAP'